### Centro Universitário da Fundação Educacional Inaciana "Padre Sabóia de Medeiros" (FEI)


*FEI's Stricto Sensu Graduate Program in Electrical Engineering*

Concentration area: ARTIFICIAL INTELLIGENCE APPLIED TO AUTOMATION AND ROBOTICS

Master's thesis student Andre Luiz Florentino

## Select the processor type

In [1]:
# Select the way to load tensorflow in order to run on CPU only or CPU + GPU

import os

opc = 0
while str(opc) not in '12':
    print()
    print("1-) CPU only")
    print("2-) CPU + GPU")

    opc = input("\nSelect the processor: ")
    if opc.isdigit():
        opc = int(opc)
    else:
        opc = 0

if opc == 1:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    
    import tensorflow as tf

else:
    import tensorflow as tf
    print(tf.__version__)

    pd = tf.config.experimental.list_physical_devices()
    for i in pd:
        print(i)
    print('------------------------------------------------------------------------------------------')


    print(tf.config.list_physical_devices('GPU'))
    # [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

    print(tf.test.is_built_with_cuda)
    # <function is_built_with_cuda at 0x000001AA24AFEC10>

    print(tf.test.gpu_device_name())
    # /device:GPU:0

    #gvd = tf.config.get_visible_devices()
    for j in tf.config.get_visible_devices():
        print(j)
    # PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
    # PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

    #physical_devices = tf.config.experimental.list_physical_devices()
    #tf.config.experimental.set_memory_growth(physical_devices[2], True)


1-) CPU only
2-) CPU + GPU

Select the processor: 1


## Check for GPU

In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [3]:
#Unload Tensorflow 
del tf

# Chapter 15: ESR (Environmental Sound Recognition)

## Import modules

In [4]:
import pyaudio
import os
import librosa
import wave

import ipywidgets      as widgets
import IPython.display as ipd

from IPython.display import display
from queue           import Queue
from threading       import Thread

In [5]:
# Globals
current_path = os.getcwd()

# For the picture names
pic_first_name = '15_ESR_'

In [6]:
# Path were the trained models and arrays are stored

path_modelsVal = os.path.join(current_path, "_ESR", "Saved_models_fold_1_validation")
path_arrays    = os.path.join(current_path, "_ESR", "Arrays")

In [7]:
# Load the ESR algorythm

from MT_ESR_evaluation_tflite import ESR_evaluation_tflite

In [8]:
cache_audio = os.path.join(current_path, '_cache_record_audio')

# Check if the folder exists, if not, create it
if not os.path.exists(cache_audio):
    os.makedirs(cache_audio)
    
CHUNK    = 1024
FORMAT   = pyaudio.paInt16
CHANNELS = 1
RATE     = 44100 # Higher sampling rate for recording live audio
RATE_ESR = 22050 # Lower sampling rate to match the predicting models

In [9]:
# Find audio device index

devices = []
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    devices.append(p.get_device_info_by_index(i))

p.terminate()

In [10]:
devices

[{'index': 0,
  'structVersion': 2,
  'name': 'Microsoft Sound Mapper - Input',
  'hostApi': 0,
  'maxInputChannels': 2,
  'maxOutputChannels': 0,
  'defaultLowInputLatency': 0.09,
  'defaultLowOutputLatency': 0.09,
  'defaultHighInputLatency': 0.18,
  'defaultHighOutputLatency': 0.18,
  'defaultSampleRate': 44100.0},
 {'index': 1,
  'structVersion': 2,
  'name': 'Microphone (WordForum USB   )',
  'hostApi': 0,
  'maxInputChannels': 1,
  'maxOutputChannels': 0,
  'defaultLowInputLatency': 0.09,
  'defaultLowOutputLatency': 0.09,
  'defaultHighInputLatency': 0.18,
  'defaultHighOutputLatency': 0.18,
  'defaultSampleRate': 44100.0},
 {'index': 2,
  'structVersion': 2,
  'name': 'Microphone Array (Intel® Smart ',
  'hostApi': 0,
  'maxInputChannels': 4,
  'maxOutputChannels': 0,
  'defaultLowInputLatency': 0.09,
  'defaultLowOutputLatency': 0.09,
  'defaultHighInputLatency': 0.18,
  'defaultHighOutputLatency': 0.18,
  'defaultSampleRate': 44100.0},
 {'index': 3,
  'structVersion': 2,
  'n

In [11]:
devices[1]

{'index': 1,
 'structVersion': 2,
 'name': 'Microphone (WordForum USB   )',
 'hostApi': 0,
 'maxInputChannels': 1,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [12]:
# Method to record the audio from the thread

RECORD_SECONDS = 1
SAMPLE_SIZE    = 2
frames         = []

def record_microphone(chunk=CHUNK):
    p = pyaudio.PyAudio()

    stream = p.open(format             = FORMAT,
                    channels           = CHANNELS,
                    rate               = RATE,
                    input              = True,
                    input_device_index = devices[1]['index'],
                    frames_per_buffer  = CHUNK)

    global frames

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [13]:
def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        
        record = Thread(target=record_microphone)
        record.start()
        
        # Here starts the ESR algorithm
        esr = Thread(target=ESR, args=(output,))
        esr.start()

In [14]:
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")

In [15]:
def play_recording(data):

    print("Playing...")

    global frames

    play=pyaudio.PyAudio()
    stream_play=play.open(format   = FORMAT,
                          channels = CHANNELS,
                          rate     = RATE,
                          output   = True)
    for data in frames: 
        stream_play.write(data)
    stream_play.stop_stream()
    stream_play.close()
    play.terminate()

In [16]:
def ESR(output):
    
    while not messages.empty():
        
        frames = recordings.get()

        # Save the audio file as .WAV for loading into Librosa
        wf = wave.open(os.path.join(cache_audio, "output.wav"), "wb")
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()

        rawdata, _  = librosa.load(os.path.join(cache_audio, 'output.wav'), sr = RATE_ESR, mono = True)
        
        print("================================================")
        print(f'Sound duration..: {librosa.get_duration(y=rawdata):2f}\n')
        ESR_evaluation_tflite([rawdata], 'CNN2D', path_modelsVal, path_arrays)

In [20]:
# Render the buttons for recording, stopping and outputing

# Pass the variables for the thread
messages   = Queue()
recordings = Queue()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

play_button = widgets.Button(
    description='Play',
    disabled=False,
    button_style='primary',
    tooltip='Play',
    icon='play'
)

output = widgets.Output()

   
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)
play_button.on_click(play_recording)

display(record_button, stop_button, play_button, output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle(), tooltip='Stop')

Button(button_style='primary', description='Play', icon='play', style=ButtonStyle(), tooltip='Play')

Output()